In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, timedelta
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import requests
import io
import time
import random
from urllib.error import HTTPError
import numpy as np

In [2]:
start_date = pd.to_datetime("2019-02-14")
end_date = pd.to_datetime("2025-02-14")
stock_data = yf.download(
                'NVDA', 
                start=start_date, 
                end=end_date + timedelta(days=1),
                progress=False  # Disable progress bar to reduce console output
            )

YF.download() has changed argument auto_adjust default to True


In [3]:
stock_data.head()

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2019-02-14,3.831802,3.860070,3.746750,3.789896,827264000
2019-02-15,3.901481,4.063402,3.878668,4.040589,1516748000
2019-02-19,3.884124,3.964713,3.871726,3.890819,551892000
2019-02-20,3.931485,3.998684,3.902226,3.913384,540988000
2019-02-21,3.862550,3.968679,3.847920,3.944131,448548000


In [6]:
fig = make_subplots(
        rows=len([1]), 
        cols=1,
    )
fig.add_trace(
                go.Line(
                    x=stock_data.index,
                    y=stock_data['Close'].values.flatten(),
                    name='ticker',
                    line=dict(width=2),
                    mode='lines',  # Explicitly set mode to lines
                    opacity=0.8    # Make slightly transparent to see markers better
                ),
                row=1, col=1
            )

/Users/humdaan/repos/llm-vs-markets/.venv/lib/python3.12/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [3]:
# Create a simple Dash plot of stock data
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Create a simple plotly figure
fig = px.line(
    x=stock_data.index, 
    y=stock_data['Close'].values.flatten(),
    # labels={'x': 'Date', 'y': 'Stock Price'},
    # title='NVIDIA Stock Price Over Time'
)

# Add some styling
fig.update_layout(
    template='plotly_white',
    xaxis=dict(
        title='Date',
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        title='Price (USD)',
        gridcolor='lightgray',
        showgrid=True
    ),
    hovermode='x unified'
)

# Display the figure
fig.show()

# # Alternative simple matplotlib plot
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# plt.plot(stock_data.index, stock_data['Close'])
# plt.title('NVIDIA Stock Price')
# plt.xlabel('Date')
# plt.ylabel('Price (USD)')
# plt.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.show()
